# Pasos a seguir.

1. Importamos las librerias necesarias.
2. Consolidamos la información de las imagenes que tienen una sola caracteristica con la siguiente estructura:
    - año: Corresponde al año del dataset de la imagen.
    - split: Indica si esta imagen se utilizó en "train, test, val".
    - nombreImg: Nombre de la imagen.
    - caract_nombre: Caracteristica de la imagen en forma de texto.
    - caract_valor: Numero de veces que se repite la caracteristica en la imagen.
    - caract_nombre_num: Caracteristica de la imagen en forma de numero.
3. Copiamos la imagen al nuevo bucket con la siguiente estructura:
    - Nombre del bucket "proyecto-agricultura-precision-mineria".
        - dataset
            - train.
                - rgb.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
                - nir.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
                - bounds.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
                - masks.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
            - test
                - rgb.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
                - nir.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
                - bounds.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.
                - masks.
                    - double_plant.
                    - weed_cluster.
                    - drydown.
                    - water.
                    - planter_skip.
                    - endrow.
                    - storm_damage.
                    - nutrient_deficiency.
                    - waterway.

In [10]:
! aws s3 cp 'C:\Users\57322\Downloads\consolidada\dataset-mineria' 's3://agriculture-vision-analitics/dataset-min/' --recursive


usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type


In [9]:
!dir

 El volumen de la unidad C no tiene etiqueta.
 El n£mero de serie del volumen es: 4AFC-256E

 Directorio de C:\Users\57322\Downloads\consolidada

27/04/2021  09:14 p.ÿm.    <DIR>          .
27/04/2021  09:14 p.ÿm.    <DIR>          ..
27/04/2021  12:41 a.ÿm.    <DIR>          .ipynb_checkpoints
09/11/2019  12:30 a.ÿm.    <DIR>          data2017_miniscale
07/02/2021  10:58 p.ÿm.     1.879.589.036 data2017_miniscale.tar
09/11/2019  12:31 a.ÿm.    <DIR>          data2018_miniscale
12/02/2021  12:34 a.ÿm.    13.662.811.864 data2018_miniscale.tar
09/11/2019  12:47 a.ÿm.    <DIR>          data2019_miniscale
12/02/2021  12:05 a.ÿm.     5.525.607.119 data2019_miniscale.tar
27/04/2021  12:18 a.ÿm.         5.069.186 dataFinalUnicos.csv
27/04/2021  10:40 a.ÿm.    <DIR>          dataset-estadistica
27/04/2021  02:49 p.ÿm.       327.004.160 dataset-estadistica.tar
27/04/2021  07:54 p.ÿm.       317.074.082 dataset-estadistica.zip
27/04/2021  12:46 a.ÿm.    <DIR>          dataset-mineria
27/04/2021  

# 1.

In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role
import json
import pandas as pd
import matplotlib.image as mpimg
from io import BytesIO
import matplotlib.pyplot as plt
from glob import glob

# 2.

In [2]:
# Verifico el rol.
role = get_execution_role()
print(role)

arn:aws:iam::280664433992:role/service-role/AmazonSageMaker-ExecutionRole-20210425T183491


In [3]:
# Verifico la región.
region = boto3.Session().region_name
print(region)

us-east-1


In [4]:
s3r = boto3.resource('s3')

In [5]:
# Leemos los archivos donde se encuentra la información de cada imagen.
BUCKET_NAME = 'agriculture-vision-analitics'
keys = ['Dataset/data2017_miniscale/field_stats.json','Dataset/data2018_miniscale/field_stats.json','Dataset/data2019_miniscale/field_stats.json']
# paths = [r'C:\Users\cese\AgricultureVision\field_stats_2017.json',r'C:\Users\cese\AgricultureVision\field_stats_2018.json',r'C:\Users\cese\AgricultureVision\field_stats_2019.json']
data = dict()
years = []
ixdata = []
for ix, keyix in enumerate (keys):
#    with open(path, 'r') as myfile:
        year=keys[ix][12:16]
        years.append(year)
        datajson=s3r.Object(BUCKET_NAME, keyix).get()['Body'].read()
        #datajson=myfile.read()
        data[year]=json.loads(datajson) 
print('Los archivos json de información fueron leidos en los diccionarios data[]...',years)

Los archivos json de información fueron leidos en los diccionarios data[]... ['2017', '2018', '2019']


In [6]:
# Cargamos el dataset para cada splits.
splits = ['train','test','val']
#dflabel_counts = pd.DataFrame(columns=['año','split','imagen','caract_grupo','caract_nombre','caract_grupo']) 
label_counts_list = []
for year in years:
    for split in splits:
        for idx_imagen, nombre_imagen in enumerate(data[year][split]):
            caract_nombre = list(data[year][split][nombre_imagen]["label_counts"].keys())
            caract_valor = list(data[year][split][nombre_imagen]["label_counts"].values())
            for nombre,valor in zip(caract_nombre,caract_valor):
                label_counts_list.append([year,split,nombre_imagen,'label_counts',nombre,valor])
dflabel_counts=pd.DataFrame(label_counts_list,columns=['año','split','imagen','caract_grupo','caract_nombre','caract_valor'])
dflabel_counts['caract_valor_onehot']=(dflabel_counts.caract_valor>0).astype(int)
dflabel_counts.head(3)

,año,split,imagen,caract_grupo,caract_nombre,caract_valor,caract_valor_onehot
0,2017,train,../datasets/data2017_miniscale/field_bounds/KF...,label_counts,planter_skip,0,0
1,2017,train,../datasets/data2017_miniscale/field_bounds/KF...,label_counts,water,0,0
2,2017,train,../datasets/data2017_miniscale/field_bounds/KF...,label_counts,weed_cluster,0,0


In [7]:
# identificamos las imagenes que tienen mas de una caracteristicas en este caso no nos interesan.
imgVariasCaract = dflabel_counts.groupby(['imagen'])['caract_valor_onehot'].sum()
imgVariasCaract = pd.DataFrame(imgVariasCaract)
imgVariasCaract = imgVariasCaract.query('caract_valor_onehot > 1')
imgVariasCaract = imgVariasCaract.reset_index()
imgVariasCaract = imgVariasCaract.imagen
imgVariasCaract = pd.DataFrame(imgVariasCaract)
print("Cantidad de imagenes que tienes mas de una caracteristica: " + str(len(imgVariasCaract)))

Cantidad de imagenes que tienes mas de una caracteristica: 5869


In [8]:
# Quitamos las imagenes que tienes mas de una caracteristica.
dataFinal = dflabel_counts.query('caract_valor_onehot == 1')
dataFinal = dataFinal.query('imagen not in @imgVariasCaract.imagen')
dataFinal.head()

,año,split,imagen,caract_grupo,caract_nombre,caract_valor,caract_valor_onehot
6,2017,train,../datasets/data2017_miniscale/field_bounds/KF...,label_counts,double_plant,1,1
11,2017,train,../datasets/data2017_miniscale/field_bounds/FM...,label_counts,weed_cluster,1,1
21,2017,train,../datasets/data2017_miniscale/field_bounds/8G...,label_counts,drydown,2,1
30,2017,train,../datasets/data2017_miniscale/field_bounds/BL...,label_counts,drydown,1,1
42,2017,train,../datasets/data2017_miniscale/field_bounds/AL...,label_counts,double_plant,6,1


In [9]:
print("RESUMEN: \n")
print("En total son: " + str(len(dflabel_counts.imagen.unique())) + " imagenes.")
print("De las cuales "+ str(len(imgVariasCaract)) + " tienen mas de 1 caracteristica.")
print("El numero de imagenes que tienen 1 sola caracteristica con las que vamos a trabajar son:" + str(len(dataFinal.imagen.unique())))

RESUMEN: 

En total son: 94986 imagenes.
De las cuales 5869 tienen mas de 1 caracteristica.
El numero de imagenes que tienen 1 sola caracteristica con las que vamos a trabajar son:89117


In [10]:
# Solo me quedo con el nombre de la imagen.
new = dataFinal["imagen"].str.split("/", n = 4, expand = True)
new_ = new[4].str.split(".png", n = 2, expand = True)
dataFinal["nombreImg"]= new_[0]
dataFinalUnicos = dataFinal[['año', 'split', 'nombreImg', 'caract_nombre', 'caract_valor']]
dataFinalUnicos.head()

,año,split,nombreImg,caract_nombre,caract_valor
6,2017,train,KFK97F7D8_3452-1417-3964-1929,double_plant,1
11,2017,train,FMWV83TKN_964-4605-1476-5117,weed_cluster,1
21,2017,train,8GCECX616_894-1569-1406-2081,drydown,2
30,2017,train,BLJ7WL94I_967-463-1479-975,drydown,1
42,2017,train,AL19NWKKB_1713-1209-2225-1721,double_plant,6


In [11]:
# Le asigno un numero a la categoria
ordinal_label = {k: i for i, k in enumerate(dataFinalUnicos['caract_nombre'].unique(), 0)}
dataFinalUnicos['caract_nombre_num'] = dataFinalUnicos['caract_nombre'].map(ordinal_label)
dataFinalUnicos.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,año,split,nombreImg,caract_nombre,caract_valor,caract_nombre_num
6,2017,train,KFK97F7D8_3452-1417-3964-1929,double_plant,1,0
11,2017,train,FMWV83TKN_964-4605-1476-5117,weed_cluster,1,1
21,2017,train,8GCECX616_894-1569-1406-2081,drydown,2,2
30,2017,train,BLJ7WL94I_967-463-1479-975,drydown,1,2
42,2017,train,AL19NWKKB_1713-1209-2225-1721,double_plant,6,0


In [12]:
print(dataFinalUnicos.caract_nombre.unique())
print(dataFinalUnicos.caract_nombre_num.unique())

['double_plant' 'weed_cluster' 'drydown' 'water' 'planter_skip' 'endrow'
 'storm_damage' 'nutrient_deficiency' 'waterway']
[0 1 2 3 4 5 6 7 8]


In [14]:
dataFinalUnicos.to_csv("dataFinalUnicos.csv", sep=',', index=False)

# 3.

In [13]:
# Creo el bucket donde voy alojar las imagenes ordenadas
bucket_name = "proyecto-agricultura-precision-mineria"
try:
    if  region == 'us-east-1':
        s3r.create_bucket(Bucket=bucket_name)
    print('Bucket S3 creado con éxito')
except Exception as e:
    print('S3 error: ',e)

Bucket S3 creado con éxito


In [15]:
# Ordenamos las imagenes de entrenamiento para la caracteristica
train_val_double_plant = dataFinalUnicos.query('caract_nombre == "double_plant" and split in ("train","val")')
train_val_weed_cluster = dataFinalUnicos.query('caract_nombre == "weed_cluster" and split in ("train","val")')
train_val_drydown = dataFinalUnicos.query('caract_nombre == "drydown" and split in ("train","val")')
train_val_water = dataFinalUnicos.query('caract_nombre == "water" and split in ("train","val")')
train_val_planter_skip = dataFinalUnicos.query('caract_nombre == "planter_skip" and split in ("train","val")')
train_val_endrow = dataFinalUnicos.query('caract_nombre == "endrow" and split in ("train","val")')
train_val_storm_damage = dataFinalUnicos.query('caract_nombre == "storm_damage" and split in ("train","val")')
train_val_nutrient_deficiency = dataFinalUnicos.query('caract_nombre == "nutrient_deficiency" and split in ("train","val")')
train_val_waterway = dataFinalUnicos.query('caract_nombre == "waterway" and split in ("train","val")')

In [21]:
total = (len(train_val_double_plant) +
      len(train_val_weed_cluster) + len(train_val_drydown) + len(train_val_water) +
      len(train_val_planter_skip) + len(train_val_endrow) + len(train_val_storm_damage) +
      len(train_val_nutrient_deficiency) + len(train_val_waterway))

In [23]:
print("Total de imagenes entrenamiento y validación: " + str(total))

Total de imagenes entrenamiento y validación: 70545


In [26]:
print("double_plant: "+str(len(train_val_double_plant)))
print("weed_cluster: "+str(len(train_val_weed_cluster)))
print("drydown: "+str(len(train_val_drydown)))
print("water: "+str(len(train_val_water)))
print("planter_skip: "+str(len(train_val_planter_skip)))
print("endrow: "+str(len(train_val_endrow)))
print("storm_damage: "+str(len(train_val_storm_damage)))
print("nutrient_deficiency: "+str(len(train_val_nutrient_deficiency)))
print("waterway: "+str(len(train_val_waterway)))

double_plant: 5391
weed_cluster: 12290
drydown: 22484
water: 2995
planter_skip: 2726
endrow: 3309
storm_damage: 411
nutrient_deficiency: 16813
waterway: 4126


In [ ]:
# imagenes categoria storm_damage para RGB.
try:
    cont = 0
    nombreDataset = train_val_storm_damage
    categoria = 'storm_damage'
    for idx, valor in enumerate(nombreDataset.nombreImg):
        anho = nombreDataset['año'].iloc[idx]
        ubicacionImg = 's3://agriculture-vision-analitics/Dataset/data{}_miniscale/field_bounds/{}.png'.format(anho,valor)
        ubicacionImgNew = 's3://proyecto-agricultura-precision-mineria/dataset/train/bounds/{}/{}.png'.format(categoria,valor)
        
        !aws s3 cp $ubicacionImg $ubicacionImgNew
        
        cont = cont+1
    print("Se cargaron" + str(cont) + " imagenes")
    
except Exception as e:
    print('Error al cargar la imagen: ',e)

copy: s3://agriculture-vision-analitics/Dataset/data2017_miniscale/field_bounds/1DQUZKY8G_1759-304-2271-816.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/1DQUZKY8G_1759-304-2271-816.png
copy: s3://agriculture-vision-analitics/Dataset/data2017_miniscale/field_bounds/1DQUZKY8G_2170-927-2682-1439.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/1DQUZKY8G_2170-927-2682-1439.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_6094-3544-6606-4056.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_6094-3544-6606-4056.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/QW9RQA76P_3819-1105-4331-1617.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/QW9RQA76P_3819-1105-4331-1617.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/2ZTIQA3AV_

copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_3126-6228-3638-6740.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_3126-6228-3638-6740.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/QW9RQA76P_4295-1227-4807-1739.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/QW9RQA76P_4295-1227-4807-1739.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_1826-6711-2338-7223.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_1826-6711-2338-7223.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_4046-5080-4558-5592.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_4046-5080-4558-5592.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/2ZTI

copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_4891-1674-5403-2186.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_4891-1674-5403-2186.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_1486-6104-1998-6616.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_1486-6104-1998-6616.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_3638-6228-4150-6740.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_3638-6228-4150-6740.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/ERUT1A634_7275-2472-7787-2984.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/ERUT1A634_7275-2472-7787-2984.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32D

copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_1017-5797-1529-6309.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_1017-5797-1529-6309.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/QW9RQA76P_2164-1303-2676-1815.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/QW9RQA76P_2164-1303-2676-1815.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/QW9RQA76P_5749-1278-6261-1790.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/QW9RQA76P_5749-1278-6261-1790.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/2ZTIQA3AV_6978-3279-7490-3791.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/2ZTIQA3AV_6978-3279-7490-3791.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/XZKR

copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/DQEV2LHCV_12620-5607-13132-6119.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/DQEV2LHCV_12620-5607-13132-6119.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/X32DPL33V_4046-3032-4558-3544.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/X32DPL33V_4046-3032-4558-3544.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/ERUT1A634_6348-5240-6860-5752.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/ERUT1A634_6348-5240-6860-5752.png
copy: s3://agriculture-vision-analitics/Dataset/data2018_miniscale/field_bounds/6MCJ8L1PM_4051-1276-4563-1788.png to s3://proyecto-agricultura-precision-mineria/dataset/train/bounds/storm_damage/6MCJ8L1PM_4051-1276-4563-1788.png
